In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = Dataset()
dr = DataReader()


In [4]:
recom = ItemKNN(dataset, topk=100)

In [5]:
train = dataset.get_train_df()
fd = dr.get_full_data()
train_sub = dataset.get_train_df_user_subset()

In [6]:
recom.compute_similarity_matrix(train_sub)  

INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 99147


func:interactions_to_sparse_matrix
 took: 1.2948768138885498 sec


In [10]:
recs = recom.recommend_multicore(interactions=train_sub, batch_size=10_000, num_cpus=10, remove_seen=False)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE
INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 4958
INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...


func:interactions_to_sparse_matrix
 took: 0.22511935234069824 sec


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 4958


func:interactions_to_sparse_matrix
 took: 0.20349788665771484 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 4958
INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...


...

INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 4957


func:interactions_to_sparse_matrix
 took: 0.07710504531860352 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
Process ForkPoolWorker-15:
Process ForkPoolWorker-18:
Process ForkPoolWorker-26:
Process ForkPoolWorker-24:
Process ForkPoolWorker-13:
Process ForkPoolWorker-16:
Process ForkPoolWorker-23:
Process ForkPoolWorker-11:
Process ForkPoolWorker-25:
Process ForkPoolWorker-19:
Process ForkPoolWorker-12:


KeyboardInterrupt: 

Process ForkPoolWorker-28:
Process ForkPoolWorker-14:
Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Process ForkPoolWorker-17:
Process ForkPoolWorker-27:
Process ForkPoolWorker-20:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/raid/home/edoardo/GitProjects/HnMChallenge/.venv/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/raid/home/edoardo/Git

In [8]:
map_at_k(rating_true=dataset.get_val_df_user_subset(), rating_pred=recs)

0.015227202906427983

In [9]:
map_at_k(rating_true=dataset.get_test_df_user_subset(), rating_pred=recs)

0.00791077573723103

In [1]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()                                               

In [15]:
sh.create_submission(recs, sub_name="TopPop_1M_rs")

In [ ]:
user_map_dict, item_map_dict = dr.get_new_raw_mapping_dict()

In [ ]:
grp_recs_df = recs.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(list)
grp_recs_df = grp_recs_df.to_frame().reset_index()
# map back to original ids
grp_recs_df[DEFAULT_USER_COL] = grp_recs_df[DEFAULT_USER_COL].apply(
    lambda x: user_map_dict.get(x)
)

In [ ]:
grp_recs_df[DEFAULT_ITEM_COL] = grp_recs_df[DEFAULT_ITEM_COL].apply(
    lambda x: " ".join(list(map(item_map_dict.get, x)))
)

In [ ]:
grp_recs_df

In [ ]:
map_at_k(rating_true=test_df_s, rating_pred=recs)

In [ ]:
map_at_k(rating_true=val_df_s, rating_pred=recs)

In [3]:
dr = DataReader()

In [4]:
dr.get_zero_interactions_recs()

,customer_id,prediction
0,00058ecf091cea1bba9d800cabac6ed1ae284202cdab68...,0751471001 0909370001 0918522001 0924243001 09...
1,000df4d2084d142416b8165bdd249bab8fea2393447aed...,0751471001 0909370001 0918522001 0924243001 09...
2,00193ff7f374dbcfecfa7fead0488e454be4918bec1ebd...,0751471001 0909370001 0918522001 0924243001 09...
3,001f00e8c1eba437ff0dbad26a9a3d49e47cbf05fff02a...,0751471001 0909370001 0918522001 0924243001 09...
4,002648d8f3b288531b24860f4a68a31d029ec5a0495c04...,0751471001 0909370001 0918522001 0924243001 09...
...,...,...
9694,ffe5801cb2a5b51d4d068322d7f8082e995f427a6f22a6...,0751471001 0909370001 0918522001 0924243001 09...
9695,ffeb3ca867aba57a312fe9d28d67dd46ef2240fe92a94c...,0751471001 0909370001 0918522001 0924243001 09...
9696,fff456fa60aac9174456c2f36ede5e0f25429a16c88a34...,0751471001 0909370001 0918522001 0924243001 09...
9697,fffa8d3cea26d4f5186472b923629b35fa28051f258030...,0751471001 0909370001 0918522001 0924243001 09...


In [11]:
zero_int_users

,customer_id,prediction
0,00058ecf091cea1bba9d800cabac6ed1ae284202cdab68...,0751471001 0909370001 0918522001 0924243001 09...
1,000df4d2084d142416b8165bdd249bab8fea2393447aed...,0751471001 0909370001 0918522001 0924243001 09...
2,00193ff7f374dbcfecfa7fead0488e454be4918bec1ebd...,0751471001 0909370001 0918522001 0924243001 09...
3,001f00e8c1eba437ff0dbad26a9a3d49e47cbf05fff02a...,0751471001 0909370001 0918522001 0924243001 09...
4,002648d8f3b288531b24860f4a68a31d029ec5a0495c04...,0751471001 0909370001 0918522001 0924243001 09...
...,...,...
9694,ffe5801cb2a5b51d4d068322d7f8082e995f427a6f22a6...,0751471001 0909370001 0918522001 0924243001 09...
9695,ffeb3ca867aba57a312fe9d28d67dd46ef2240fe92a94c...,0751471001 0909370001 0918522001 0924243001 09...
9696,fff456fa60aac9174456c2f36ede5e0f25429a16c88a34...,0751471001 0909370001 0918522001 0924243001 09...
9697,fffa8d3cea26d4f5186472b923629b35fa28051f258030...,0751471001 0909370001 0918522001 0924243001 09...
